In [ ]:
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
import cmaps
import warnings
import sacpy as scp
from cartopy.util import add_cyclic_point
import matplotlib as mpl
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_CBR_coldhot = mpl.colors.ListedColormap(get_cmap_pr(18,cmaps.CBR_coldhot,method=2))
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
uwnd = np.load("/public/home/songqh/project/ISM_impact_Antarctica/data/uwnd_200hPa_1979-2024.npy")
lat = np.load("/public/home/songqh/project/ISM_impact_Antarctica/data/lat.npy")
lon = np.load("/public/home/songqh/project/ISM_impact_Antarctica/data/lon.npy")
dy = 1.0  
uwnd_grad_y = np.gradient(uwnd, dy, axis=1)
uwnd_grad_y_mean = np.nanmean(uwnd_grad_y,axis=0)

In [ ]:
file1 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1979-2024.nc")
precip_index = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),28:10,70:100].groupby('time.year').mean('time').rename({'year':'time'}).mean(('longitude','latitude'))*1000
precip_index = (precip_index - np.mean(precip_index,axis=0))/np.std(precip_index)

In [ ]:
LRG1 = scp.LinReg(precip_index,uwnd_grad_y[:])

In [ ]:
cycle_uwnd_grad_y_mean,cycle_lon = add_cyclic_point(uwnd_grad_y_mean, coord=lon)
cycle_slope,cycle_lon = add_cyclic_point(LRG1.slope, coord=lon)
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211,projection = ccrs.PlateCarree(central_longitude=150))
ax1.draw_map([-60,280,-90,30],latspec=30,res=110)
ax1.set_title("Mean ∂u/∂y",fontsize=20,loc='center')
ax1.set_title("a",fontsize=24,weight='bold',loc='left')
cf1 = ax1.contourf(cycle_lon,lat,cycle_uwnd_grad_y_mean, zorder=2,levels=np.linspace(-8,8,11),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.02)
cb.ax.set_colorbar(minor='off')

ax2 = fig.add_subplot(212,projection = ccrs.PlateCarree(central_longitude=150))
ax2.draw_map([-60,280,-90,30],latspec=30)
ax2.set_title("∂u/∂y reg. on ISM rainfall index",fontsize=20,loc='center')
ax2.set_title("b",fontsize=24,weight='bold',loc='left')
cf2 = ax2.contourf(cycle_lon,lat,cycle_slope, zorder=2,levels=np.linspace(-0.5,0.5,11),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
cb = fig.colorbar(cf2,orientation='vertical',shrink=0.7,pad=0.02)
cb.ax.set_colorbar(minor='off')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/npj_figure/support-5.jpeg",dpi=600, bbox_inches='tight')